In [19]:
import pandas as pd
import numpy as np

with open("./data/causal_inference.txt", 'r') as f:
    text = f.readlines()

res = []
for t in text:
    res.append(list(map(int, t.strip('\n').split(" ")[:-1]))) # 最後空文字が入っている。
#     print(t.strip('\n').split(" ")[:-1])

In [20]:
ds = pd.DataFrame(res)
ds.columns = ["arm", "reward"] + [f"feat_{i}" for i in range(100)]

In [21]:
X = ds.loc[:, "feat_0":].values
y = ds.loc[:, "reward"].values
arms = ds.loc[:,"arm"].values

In [22]:
from src.Arms import ContextualArms
Arms = ContextualArms(
    n_action=10, 
    max_n_sim=100, 
    X=X, 
    y=y,
    ds=ds,
    arms=arms, 
    n_features=X.shape[1], 
    is_logit=False)

In [23]:
Arms.draw()
Arms.get_reward(1)

26.782077393075355

In [24]:
Arms.best_arms()

(2, 26.782077393075355)

In [25]:
d = 100
sigma=1; sigma_0=1
A_inv =sigma_0/sigma * np.eye(d); A_inv
b_t = np.zeros((d, 1))
alpha = 0.05
n_arms = 10
n_sim = 1000
# history = []
history = dict(chosen_arms=[],rewards=[], regrets=[])

In [ ]:
mu_hat = np.zeros((n_arms, 1))

# LinUCBアルゴリズム
for t in range(1, n_sim):
    
    # 3.
    theta = A_inv@b_t 
    # calc alpha_t
    alpha_t = alpha*np.sqrt(np.log(t))
    # 4. A[t]はa_itの転置ver
    
    # 各行動で以下のUCB scoreを計算する。
    # 各アームの特徴量を引く
    # shape : (10, 1, 100) 10 arms, (1, 100)の特徴量ベクトル
    # 1次元目にアーム１から10までの特徴量を順番に格納している。
    A_ = Arms.draw()
    for i in range(n_arms):
        mu_hat[i] = np.dot(A_[i], theta) + alpha_t*sigma*np.sqrt(A_[i]@A_inv@A_[i].T)
    i_star = mu_hat.argmax(0)[0]
    history["chosen_arms"].append(i_star)
    
    #5. スコア最大の行動 i*　を選択して報酬を観測する。
    reward = Arms.get_reward(index=i_star)
    history["rewards"].append(reward)
    # calculate regret
    num_arm, best_reward = Arms.best_arms()
    history["regrets"].append(best_reward - reward)

    a_is = A_[i_star, np.newaxis]
    #6.  A-inverse/ bの更新をする。
    second_term = (A_inv@a_is.T@a_is@A_inv) / (1+a_is@A_inv@a_is.T)
    A_inv = A_inv - second_term
    b_t += a_is.T*reward    

In [16]:
A_[i_star].T

array([ 1,  0, 31,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 14,  0, 22,  0,  0,  2,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, 38,  0,  0,  1,  0,  6, 33,  1,  0, 10,  0,  8,
        3,  0,  1,  4,  0,  0,  0,  0,  1,  0,  1,  1,  0, 56,  2,  0,  0,
        4,  0,  0,  0,  1,  0, 20,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,
        0,  0,  5,  0,  1,  0,  0,  6,  0,  1,  2,  0,  0,  0, 12])